Very Fast Decision Trees
========================

General Idea:

*  Hoeffding bounds use the notion of PAC learning (Probably Approximately Correct) to yield an "idea" that there is an appropriate split
   *  this bound relies on knowing the _range_ of the metric under question, and the number of instances seen only! This is what makes it "agnostic" to different metrics
   
$$e = \sqrt{\frac{R^2 \log(1/\delta)}{2n}}$$

where $R$ is the range of the metric and $1-\delta$ is the probability that we have the "true mean" of the target metric. If we use gini impurity for a binary classification problem, then $R = 0.5$ (exercise forthe reader).

_**Aside**: the idea of something being PAC learnable is very important in Learning theory, and is what many consider to be the differentiator between classical statistics and (modern) machine learning theory. These notes contains some other bounds typically studied in these classes (both statistics and machine learning): https://www.stat.purdue.edu/~jianzhan/STAT598Y/NOTES/slt03.pdf_

We can use these bounds in the following way:

```
In a set of proposed splits for a decision tree node:
1.  Calculate the <proposed> metric for every proposed split
2.  Calculate the difference between the top 2 proposed split
3.  Then:
    a. If the difference of the top 2 proposed split is less than 'e' then set this as a valid decision node split
    b. otherwise continue training
```
----

How do we propose splits?

In VFDT the split proposal uses Naive Bayes models, due to the independence assumptions. Unlike CART style models, this means that each feature only has one candidate split. 

In [2]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from skmultiflow.data import SEAGenerator


stream = SEAGenerator()
model = GaussianNB(priors=[0.5, 0.5])


def find_split(
    model, n_seen=0, proposal_scores=None, delta=0.05, min_seen=100, *args, **kwargs
):
    """
    Attempts to find splits.

    Returns:
    {
        'is_split' - bool,
        'n_seen' - int
        'proposal_scores' - array
        'model' - naive bayes model
        'split info' - None or {feat_idx:val}
    }
    """
    X, y = stream.next_sample(5)
    hoeffding_bound = lambda n: np.sqrt(0.25 * np.log(1 / delta) / (2 * n))
    if n_seen == 0:
        model.fit(X, y)
    else:
        model.partial_fit(X, y)
    proposed_splits = np.mean(model.theta_, 0)
    n_seen += X.shape[0]

    if proposal_scores is None:
        proposal_scores = np.zeros(X.shape[1])
    for feat_idx in range(X.shape[1]):
        y_indx = X[:, feat_idx] < proposed_splits[feat_idx]
        proposal_scores[feat_idx] += np.sum(y[y_indx])

    impurity = [
        2 * (x / n_seen) * (1 - x / n_seen) for x in proposal_scores
    ]  # this isn't 100% correct, I'm just lazy...
    vals = sorted(impurity.copy())[:2]
    is_split = False
    split_info = None
    print("Seen: ", n_seen, ". Minimum before making a decision: ", min_seen)
    print(
        "\tCurrently diff between top two scores are ",
        np.abs(vals[0] - vals[1]),
        ". hoeffding bound: ",
        hoeffding_bound(n_seen),
    )
    if np.abs(vals[0] - vals[1]) < hoeffding_bound(n_seen) and n_seen >= min_seen:
        is_split = True
        split_info = {
            "feat": np.argmin(impurity),
            "val": proposal_scores[np.argmin(impurity)],
            "impurity": np.min(impurity),
        }

    return dict(
        is_split=is_split,
        model=model,
        n_seen=n_seen,
        min_seen=min_seen,
        proposal_scores=proposal_scores,
        split_info=split_info,
    )


split_not_found = True
info = {
    "is_split": False,
    "model": model,
    "n_seen": 0,
    "proposal_scores": None,
    "split_info": None,
}
while not info["is_split"]:
    info = find_split(**info)

import pprint
pprint.pprint(info)

Seen:  5 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.0 . hoeffding bound:  0.2736664152555987
Seen:  10 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.0 . hoeffding bound:  0.19351137801024748
Seen:  15 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.08888888888888893 . hoeffding bound:  0.1580013785159798
Seen:  20 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.19499999999999998 . hoeffding bound:  0.13683320762779935
Seen:  25 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.05120000000000008 . hoeffding bound:  0.12238734153404082
Seen:  30 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.04222222222222227 . hoeffding bound:  0.1117238461854718
Seen:  35 . Minimum before making a decision:  100
	Currently diff between top two scores are  0.0 . hoeff